## Working with carto for web-based interaction of mobility

In [1]:
# make display wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:88% !important; }</style>"))

In [1]:
# read in packages
%matplotlib inline
import pandas as pd
import json
import geopandas as gpd
import numpy as np
import os
from cartoframes.auth import Credentials, set_default_credentials # unsure of exact difference
from cartoframes.data.services import Geocoding, Isolines # these may not be needed
from cartoframes.data.observatory import Enrichment
from cartoframes.viz import Map, Layer, widget, basemaps, color_continuous_style, size_continuous_style # to create a simple choropleth
from cartoframes.viz.widgets import animation_widget

In [2]:
# allow max rows and colums to be displayed
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r'C:\Users\steve\GitHub\rp-covid-migration')

In [3]:
# connect to carto authorization using credentials
username = os.environ.get('Carto_user')
pw = os.environ.get('Carto_Auth')
credentials = Credentials(username = 'dcpregional', api_key = pw)
credentials.save('carto_creds.json')

User credentials for `dcpregional` were successfully saved to `carto_creds.json`


In [4]:
# read in subplace values by day
ny_to_subpl = pd.read_csv(r'data\dest_mnxsubpl_daily_01-09-2020.csv')
ny_to_subpl.head()

,SubPlace,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,1/7/20,1/8/20,1/9/20,1/10/20,1/11/20,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,U0900104720,6.228492,0.000000,1.577909,4.752400,1.568923,0.000000,0.000000,0.000000,0.000000,1.657001,1.681435,0.000000,5.248976,0.000000,0.000000,3.883118,0.000000,16.379346,5.971219,6.017089,0.000000,0.000000,0.000000,0.000000,4.082966,8.158603,0.000000,0.000000,0.000000,0.000000,0.000000,4.784765,0.000000,0.000000,1.661847,1.722653,1.722386,1.669561,1.666195,0.000000,1.627922,0.000000,0.000000,0.000000,0.000000,9.259602,15.230552,1.923151,0.000000,1.892792,1.830798,3.746651,9.477954,0.000000,0.000000,0.000000,0.000000,1.804110,1.924446,11.814047,9.820289,1.613294,2.974730,2.977077,2.975615,1.520612,11.034918,3.214349,1.569588,3.459011,3.567097,5.245030,6.815703,5.644296,7.315690,3.732666,3.736781,3.779504,5.619661,5.849550,10.454346,2.073269,1.949128,3.901906,3.775865,2.049096,8.248443,10.526759,2.072496,13.960353,12.037316,0.000000,3.508710,0.000000,0.000000,3.713193,0.000000,0.000000,10.233110,2.026384,2.072066,4.160945,0.000000,4.278166,0.000000,0.000000,4.341880,0.000000,0.000000,2.187657,0.000000,0.000000,4.164931,0.000000,0.000000,0.000000,10.743446,4.241152,2.106327,4.339336,0.000000,0.000000,0.000000,2.384131,2.332416,0.000000,0.000000,0.000000,2.407029,2.335357,2.303404,4.617338,4.635854,2.355935,2.337268,6.994964,11.859583,2.191829,2.182739,2.181453,2.192454,0.000000,4.162418,10.460470,6.366588,6.199372,2.077404,2.200607,2.210189,2.114523,4.058853,2.014748,0.000000,0.000000,0.000000,0.000000,12.507296,12.596309,6.316454,2.069922,2.089558,2.065006,0.000000,2.105352,2.096612,6.364427,0.000000,2.287335,2.286028,2.345546,11.782449,7.002311,7.395720,0.000000,0.000000,0.000000,2.318626,2.310750,11.827601,12.115925,2.332851,2.265211,0.000000,2.369893,2.324500,2.386749,2.356823,7.103282,2.275520,6.719075,2.246282,0.000000,7.050197,0.000000,2.266546,0.000000,2.343677,0.000000,4.789272,0.000000,0.000000,0.000000,4.425072,0.000000,0.000000,2.1

In [5]:
# alter file, convert geoid field to string
ny_to_subpl = ny_to_subpl.rename(columns={'SubPlace':'id'})
ny_to_subpl['id'] = ny_to_subpl['id'].str[1:]
ny_to_subpl.head()

,id,1/1/20,1/2/20,1/3/20,1/4/20,1/5/20,1/6/20,1/7/20,1/8/20,1/9/20,1/10/20,1/11/20,1/12/20,1/13/20,1/14/20,1/15/20,1/16/20,1/17/20,1/18/20,1/19/20,1/20/20,1/21/20,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20
0,0900104720,6.228492,0.000000,1.577909,4.752400,1.568923,0.000000,0.000000,0.000000,0.000000,1.657001,1.681435,0.000000,5.248976,0.000000,0.000000,3.883118,0.000000,16.379346,5.971219,6.017089,0.000000,0.000000,0.000000,0.000000,4.082966,8.158603,0.000000,0.000000,0.000000,0.000000,0.000000,4.784765,0.000000,0.000000,1.661847,1.722653,1.722386,1.669561,1.666195,0.000000,1.627922,0.000000,0.000000,0.000000,0.000000,9.259602,15.230552,1.923151,0.000000,1.892792,1.830798,3.746651,9.477954,0.000000,0.000000,0.000000,0.000000,1.804110,1.924446,11.814047,9.820289,1.613294,2.974730,2.977077,2.975615,1.520612,11.034918,3.214349,1.569588,3.459011,3.567097,5.245030,6.815703,5.644296,7.315690,3.732666,3.736781,3.779504,5.619661,5.849550,10.454346,2.073269,1.949128,3.901906,3.775865,2.049096,8.248443,10.526759,2.072496,13.960353,12.037316,0.000000,3.508710,0.000000,0.000000,3.713193,0.000000,0.000000,10.233110,2.026384,2.072066,4.160945,0.000000,4.278166,0.000000,0.000000,4.341880,0.000000,0.000000,2.187657,0.000000,0.000000,4.164931,0.000000,0.000000,0.000000,10.743446,4.241152,2.106327,4.339336,0.000000,0.000000,0.000000,2.384131,2.332416,0.000000,0.000000,0.000000,2.407029,2.335357,2.303404,4.617338,4.635854,2.355935,2.337268,6.994964,11.859583,2.191829,2.182739,2.181453,2.192454,0.000000,4.162418,10.460470,6.366588,6.199372,2.077404,2.200607,2.210189,2.114523,4.058853,2.014748,0.000000,0.000000,0.000000,0.000000,12.507296,12.596309,6.316454,2.069922,2.089558,2.065006,0.000000,2.105352,2.096612,6.364427,0.000000,2.287335,2.286028,2.345546,11.782449,7.002311,7.395720,0.000000,0.000000,0.000000,2.318626,2.310750,11.827601,12.115925,2.332851,2.265211,0.000000,2.369893,2.324500,2.386749,2.356823,7.103282,2.275520,6.719075,2.246282,0.000000,7.050197,0.000000,2.266546,0.000000,2.343677,0.000000,4.789272,0.000000,0.000000,0.000000,4.425072,0.000000,0.000000,2.105440,6

In [6]:
# read in geojson
subplaces1 = r'data\shapefiles\nycmetro_subplace.geojson' # used to convert to geodataframe 
subplaces = json.load(open('data/shapefiles/nycmetro_subplace.geojson','r')) # to view the raw json data
#check gdf
gdf = gpd.read_file(subplaces1)

In [7]:
gdf.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geometry
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,900946940.0,-234.0,"MULTIPOLYGON (((-73.07011 41.51430, -73.07154 ..."
1,2,09,009,47535,00213462,0900947535,Milford,Milford town,43,T5,G4040,720,71950,None,C,57442071.0,10218658.0,+41.2250861,-073.0611101,None,None,None,None,0.481270,0.007266,67660729.0,26.12381,900947535.0,-1743.0,"MULTIPOLYGON (((-72.99205 41.24491, -72.99209 ..."
2,3,09,009,58300,00213486,0900958300,Oxford,Oxford town,43,T1,G4040,720,71950,None,A,84803121.0,1531057.0,+41.4440006,-073.1479992,None,None,None,None,0.439399,0.009300,86334178.0,33.33363,900958300.0,248.0,"MULTIPOLYGON (((-73.10423 41.48940, -73.10423 ..."
3,4,09,009,00000,00000000,0900900000,County subdivisions not defined,County subdivisions not defined,00,Z9,G4040,None,None,None,F,0.0,599104136.0,+41.1874659,-072.8153339,None,None,None,None,1.700000,0.064300,599104136.0,231.31411,900900000.0,NaN,"MULTIPOLYGON (((-73.10455 41.17305, -73.10510 ..."
4,5,09,009,44560,00213454,0900944560,Madison,Madison town,43,T1,G4040,720,75700,None,A,93622105.0,1251401.0,+41.3444810,-072.6245213,None,None,None,None,0.739953,0.010206,94873506.0,36.63066,900944560.0,-218.0,"MULTIPOLYGON (((-72.53651 41.25461, -72.53654 ..."


In [8]:
gdf.to_crs("EPSG:4326") # gdf.to_crs(epsg=3395) would also work
gdf['geom'] = gdf.centroid # convert the polygon to centroid

gdf = gdf.set_geometry('geom')
# remove former geometry column
gdf = gdf.drop(['geometry'], axis = 1)
gdf.head()

C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geom
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.321040,0.005154,47773634.0,18.44540,900946940.0,-234.0,POINT (-73.12283 41.52707)
1,2,09,009,47535,00213462,0900947535,Milford,Milford town,43,T5,G4040,720,71950,None,C,57442071.0,10218658.0,+41.2250861,-073.0611101,None,None,None,None,0.481270,0.007266,67660729.0,26.12381,900947535.0,-1743.0,POINT (-73.06185 41.22575)
2,3,09,009,58300,00213486,0900958300,Oxford,Oxford town,43,T1,G4040,720,71950,None,A,84803121.0,1531057.0,+41.4440006,-073.1479992,None,None,None,None,0.439399,0.009300,86334178.0,33.33363,900958300.0,248.0,POINT (-73.13503 41.43129)
3,4,09,009,00000,00000000,0900900000,County subdivisions not defined,County subdivisions not defined,00,Z9,G4040,None,None,None,F,0.0,599104136.0,+41.1874659,-072.8153339,None,None,None,None,1.700000,0.064300,599104136.0,231.31411,900900000.0,NaN,POINT (-72.79470 41.25792)
4,5,09,009,44560,00213454,0900944560,Madison,Madison town,43,T1,G4040,720,75700,None,A,93622105.0,1251401.0,+41.3444810,-072.6245213,None,None,None,None,0.739953,0.010206,94873506.0,36.63066,900944560.0,-218.0,POINT (-72.62809 41.34013)


In [10]:
gdf.to_file("data/shapefiles/nyc_subplace_centroid.geojson", driver="GeoJSON")

data/shapefiles/nyc_subplace_centroid.geojson: No such file or directory
driver GeoJSON does not support creation option ENCODING


In [11]:
# view the json file and properties
subplaces['features'][0]['properties']

{'OBJECTID': 1,
 'STATEFP': '09',
 'COUNTYFP': '009',
 'COUSUBFP': '46940',
 'COUSUBNS': '00213459',
 'GEOID': '0900946940',
 'NAME': 'Middlebury',
 'NAMELSAD': 'Middlebury town',
 'LSAD': '43',
 'CLASSFP': 'T1',
 'MTFCC': 'G4040',
 'CNECTAFP': '720',
 'NECTAFP': '78700',
 'NCTADVFP': None,
 'FUNCSTAT': 'A',
 'ALAND': 45986818.0,
 'AWATER': 1786816.0,
 'INTPTLAT': '+41.5246912',
 'INTPTLON': '-073.1230162',
 'PLACEFP': None,
 'PLACENS': None,
 'PCICBSA': None,
 'PCINECTA': None,
 'Shape_Leng': 0.321039511792,
 'Shape_Area': 0.0051538166243,
 'ATOTAL': 47773634.0,
 'ATOTAL_mi': 18.4454,
 'geoid2': 900946940.0,
 'TotIndust': -234.0}

In [10]:
# cleaning dataframe for CARTO export
subplace_clean = df
subplace_clean = subplace_clean.rename(columns={'Subplace':'id'})
subplace_clean['id'] = subplace_clean['id'].str[0:]
subplace_clean.to_csv(r'data\subplaces_daily.csv')
subplace_clean.head()

NameError: name 'df' is not defined

In [23]:
subplace_clean.dtypes
# check the data types prior to viewing within CARTO

id                  object
variable    datetime64[ns]
value              float64
dtype: object

In [16]:
#transform table for animation, select only march - may
list_date = ny_to_subpl.columns.to_list()
#list_date = [x for x in list_date if x[0] == '3' or x[0] == '4' or x[0] == '5'  x[0] == '5']

#pick only march 1 - march 20, because my animation won't render for more days
#remove this line if we can get 3 months to work
list_date = list_date[1:]  

df = pd.melt(ny_to_subpl, id_vars=['id'],value_vars = list_date)
#df = df.rename(columns={'SubPlace':'id'}) #rename for consistent geojson join
#df['id'] = df['id'].str[1:] #clean the id for geojson join
df['variable'] = pd.to_datetime(df['variable'])
#dff = df.drop(df.value > 10)
#dff = df.drop(df[df.value <= 15.0].index)
#df = df.drop(df[df.score < 50].index)
df.head(10)

,id,variable,value
0,0900104720,2020-01-01,6.228492
1,0900108070,2020-01-01,63.842045
2,0900108980,2020-01-01,12.456984
3,0900118500,2020-01-01,29.585338
4,0900118850,2020-01-01,32.699584
5,0900123890,2020-01-01,1.557123
6,0900126620,2020-01-01,57.613553
7,0900133620,2020-01-01,105.884368
8,0900148620,2020-01-01,3.114246
9,0900150580,2020-01-01,15.571231


In [17]:
# merge the data frame (long)
#dff = pd.merge(gdf, df, left_on="GEOID", right_on="id", how="inner")
#dff.head(10)
#dff.dtypes
# merge the data frame (wide)
df_centroid = pd.merge(gdf, df, left_on="GEOID", right_on="id", how="inner")
df_centroid.head()

,OBJECTID,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,PLACEFP,PLACENS,PCICBSA,PCINECTA,Shape_Leng,Shape_Area,ATOTAL,ATOTAL_mi,geoid2,TotIndust,geom,id,variable,value
0,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-01,1.557123
1,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-02,1.589017
2,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-03,1.577909
3,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-04,1.584133
4,1,09,009,46940,00213459,0900946940,Middlebury,Middlebury town,43,T1,G4040,720,78700,None,A,45986818.0,1786816.0,+41.5246912,-073.1230162,None,None,None,None,0.32104,0.005154,47773634.0,18.4454,900946940.0,-234.0,POINT (-73.12283 41.52707),0900946940,2020-01-05,1.568923


In [15]:
len(df_centroid)


In [18]:
df_centroid.to_csv(r'C:\Users\steve\GitHub\rp-covid-migration\data\values_cntr_jansep.csv')

In [62]:
#dff.to_csv(r'C:\Users\steve\GitHub\rp-covid-migration\data\carto_time_subplaces.csv')
df_wide.to_csv(r'C:\Users\steve\GitHub\rp-covid-migration\data\subplaces_wide.csv')

In [44]:
# trying to view kepler in notebook
from keplergl import KeplerGl
map1 = KeplerGl(height = 500)
#map1
with open('data\carto_time_subplaces.csv', 'r') as file:
    dataframe = file.read()
map1.add_data(data = dataframe, name = 'data')

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [ ]:
df_centroid.head()

In [14]:
# using cartoframes to view data in notebook
Map(
     Layer(df_centroid, color_continuous_style(
             'value',
             range_min = 0,
             range_max = 1000,
             palette = 'teal',
             stroke_color = 'gray', stroke_width = 0.6, opacity = 0.8),
             default_widget = True,
             title = 'Total Movement out of NYC, March 19th', encode_data = False),
#     Layer(dff, color_continuous_style(
#             'value',
#              range_min = 0,
#             range_max = 2000,
#             palette = 'teal',
#             stroke_color = 'gray', stroke_width = 0.6, opacity = 0.8),
#            widgets = [
#                animation_widget(
#                    title = 'Date',
#                    description = 'rabblerabblerabble'),
          basemaps.darkmatter
          )

NameError: name 'df_centroid' is not defined